<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;"><b>Treball de Final de Màster </b></p>
<p style="margin: 0; text-align:right;"><b>ANÀLISI DEL DIA MUNDIAL DE LES MALALTIES MINORITÀRIES.</b>.</p>
    <p style="margin: 0; text-align:right;"><b>(Font de dades: TWITTER)</b></p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PAC 3: Disseny i Implementació del TFM.

# PROCÉS DE CAPTACIÓ DE DADES
<p style="color:#0000FF">
En aquest document, veurem com s'ha implementat el procés de captació de dades de la xarxa twitter.<br>
    S'ha utilitzat un script en llenguatge Python, adaptant a les nostres necessitats els exemples oferts al seu GitHub: 
</p>

<div style="text-align:right">
    <span style="font-family:calibri;font-size:8;color:white; background-color:Crimson; text-align:right; margin-left: 0.5cm;padding:0.3cm"> Autor: </span>
    <span style="font-family:calibri;font-size:8;color:white; background-color:RoyalBlue; text-align:center;  margin-right: 0.5cm; padding:0.3cm"><b> Joaquim de Dalmases Juanet </b></span>
</div>

In [ ]:
"""
Created on Tue Feb 11 17:20:51 2020

@author: Quim Dalmases
Projecte: Estudi Dia Mundial de les Malalties Rares
Objectiu: Procés de recollida de dades en Twitter.
Implementació: Streaming usant la llibreria Tweepy.
Framework: Spyder (Anaconda Framework)
Codi: Python v 3.8

"""
import tweepy
import codecs
import json
# Situem les claus de connexió en un modul apart per privacitat.
from secret1 import CONSUMER_KEY_KEY, CONSUMER_SECRET_KEY, ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET_KEY;
from datetime import datetime

# Funció que definiex la validació de les claus per accedir a poder captar els tuits.
def get_auth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY_KEY, CONSUMER_SECRET_KEY)
    auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET_KEY)
    return auth

# Classe que implementa el flus o stream on es reben els tuits per després ser emmagatzemats.
class MyStreamListener(tweepy.StreamListener):
    # Event que desencadena la captació i emmagatzematge en el mateix directori de l'script.
    def on_data(self, data):
        try:
            # Obtenció de la data en que es rep el tuit.
            now=datetime.now()
            # Definició del format de visualització d'aquesta data.
            data_m = now.strftime("%Y/%m/%d - %H:%M:%S")
            print(data_m)
            data_actual = now.strftime("%Y_%m_%d")
            # obtenció de l'estructura en json de dades del tuit.
            tweet = json.loads(data)
            # Extracció de cada camp d'interés del tuit, però per visualitzar
            # per pantalla i monitoritzar el procés.
            #ID únic del tuit
            id_tweet = tweet.get('id_str')
            # data de creació pròpia del tuit.
            data_tweet = str(tweet.get('created_at'))
            # Nom d'usuari
            user_tweet_name = tweet.get('user').get('name')
            # Àlies de l'usuari
            user_tweet_screen_name = tweet.get('user').get('screen_name')
            # Inspecció del camp 'truncated' per saber si la longitud es de 140 o de 280 caràcters
            # i obtenir el contingut del camp 'text' o del camp 'extended_tweet.full_text'.
            tweet_truncat=tweet.get("truncated")
            if tweet_truncat:
                text_tweet=tweet.get("extended_tweet").get("full_text")
            else:
                text_tweet = tweet.get('text')
            # Inspecció del tuit per saber si és un retuit.    
            retweet="False"
            if tweet.get("retweeted_status") is not None:
                retweet = "True"
                if tweet.get("retweeted_status").get("truncated"):
                    text_tweet=tweet.get("retweeted_status").get("extended_tweet").get("full_text")
                else:
                    text_tweet = tweet.get("retweeted_status").get('text')
                    
            if tweet.get("retweet_count")>0 or tweet.get("is_quote_status"):
                retweet = "True"
            else:
                retweet = "False"
            # Impressió per pantalla de les dades descriptives del tuit
            # per la seva monitorització.
            print("------------------------------------------------------")
            regTweet=""
            regTweet = "Tweet ID: " + id_tweet
            regTweet = regTweet + "\nData: " + data_tweet 
            regTweet = regTweet + "\nUser_name: " + user_tweet_name 
            regTweet = regTweet + "\nUser_screen_name: " + user_tweet_screen_name
            regTweet = regTweet + "\nTweet:\n" + text_tweet
            regTweet = regTweet + "\nReTweet: " + retweet

            print(regTweet)
            # Emmagatzematze a disc en el fitxer de notació: "TWEETS_RAW_(data).txt"
            # Exemple: TWEETS_RAW_2020_02_13.txt
            with codecs.open("TWEETS_RAW_"+ data_actual +".txt", "a", encoding="utf-8") as myfile:
                #json.dump(data, myfile, ensure_ascii=False)
                myfile.write(data)
                myfile.close()
            # Emmagatzematze a disc d'una selecció de dades, en el fitxer de notació: "TWEETS_(data).txt"
            # Exemple: TWEETS_2020_02_13.txt
            with codecs.open("TWEETS_"+ data_actual +".txt", "a", encoding="utf-8") as myfile1:
                registre='{"data": "' + data_tweet + '",\n' 
                registre = registre + '"name": "' + user_tweet_name + '",\n' 
                registre = registre + '"screen_name": "' + user_tweet_screen_name + '",\n'
                registre = registre + '"text": "' + text_tweet + '",\n'
                registre = registre + '"ReTweet: "' + retweet + '"},\n'
                myfile1.write(registre)
                myfile1.close()
            
        except Exception as e:
            print(data_m)
            print("ERROR: {}".format(e))
        finally:
            return True  # Resta l'script en execució permanent.      

# Codi principal de l'script        

print(" ----- Twitter Procés Captació ----- ")

# Obtenim tuits usant tweepy
auth = get_auth()  # Recuperem un objecte 'auth' usant la funció 'get_auth':
api = tweepy.API(auth)  # Construim un objecte d'API.

# Connecta i escolta el flux de dades o 'stream'
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)

print(">> Capturant tweets sobre:\n#DiaMundialEnfermedadesRaras\n#DíaMundialEnfermedadesRaras\n \
#SomosFEDER\n#EnfermedadesRaras\n#DMEnfermedadesRaras2020\n#DM2020")

myStream.filter(track=['#DiaMundialEnfermedadesRaras,#DíaMundialEnfermedadesRaras,#SomosFEDER, \
#EnfermedadesRaras,#DMEnfermedadesRaras2020,#DM2020'])
    

In [ ]:
 ----- Twitter Procés Captació ----- 
>> Capturant tweets sobre:
#DiaMundialEnfermedadesRaras
#DíaMundialEnfermedadesRaras
 #SomosFEDER
#EnfermedadesRaras
#DMEnfermedadesRaras2020
#DM2020
2020/05/05 - 10:01:04
------------------------------------------------------
Tweet ID: 1257581034075553793
Data: Tue May 05 08:00:57 +0000 2020
User_name: RarasNoInvisibles
User_screen_name: NoInvisibles
Tweet:
📢🔜 Cita importante hoy a las 17 h:

📲 #Webinar de @Share4Rare_es sobre diagnóstico genético en #EnfermedadesRaras con @GMarfanyN y @LluisMontoliu 

Registro gratuito obligatorio: https://t.co/fubY1Oy9dc

☕️¡Nunca un café virtual será tan didáctico! ¡Te esperamos! https://t.co/gpHry1wDWN
ReTweet: False